
# SSH tight-binding model using the PythTB package

__[PythTB](http://www.physics.rutgers.edu/pythtb/index.html)__

### Import packages for Python

In [ ]:
# Copyright under GNU General Public License 2010, 2012, 2016
# by Sinisa Coh and David Vanderbilt (see gpl-pythtb.txt)

from pythtb import * # import TB model class
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

### Define basic properties of the model

positions of orbitals in reduced coordinates (in units of the lattice vectors)

In [ ]:
# define lattice vectors
lat=[[1]]
# define coordinates of orbitals
orb=[[0.25],[0.75]]

### Create the model

In [ ]:
# create the SSH model
my_model=tb_model(1,1,lat,orb)

# set model parameters
v=0.5
w=1.0

# set on-site energies
my_model.set_onsite([0,0])
# set hoppings (one for each connected pair of orbitals)
# (amplitude, i, j, [lattice vector to cell containing j])
my_model.set_hop(v, 0, 1, [0])
my_model.set_hop(w, 1, 0, [1])

# print tight-binding model
my_model.display()

### Visualize the model

In [ ]:
%matplotlib inline
my_model.visualize(0)

### Prepare the path in the Brillouin zone

nodes in reduced coordiantes (in units of the reciprocal lattice vectors)

In [ ]:
# generate list of k-points following a segmented path in the BZ
# list of nodes (high-symmetry points) that will be connected
path=[[-0.5],[0.0],[0.5]]
# labels of the nodes
label=(r'$-X$',r'$\Gamma $',r'$X$')
# total number of interpolated k-points along the path
nk=101

# call function k_path to construct the actual path
(k_vec,k_dist,k_node)=my_model.k_path(path,nk)
# inputs:
#   path, nk: see above
#   my_model: the pythtb model
# outputs:
#   k_vec: list of interpolated k-points
#   k_dist: horizontal axis position of each k-point in the list
#   k_node: horizontal axis position of each original node

### Solve for the band structure

In [ ]:
# obtain eigenvalues and eigenvectors
(evals_bnd,evecs_bnd)=my_model.solve_all(k_vec,eig_vectors=True)

### Plot the band structure

In [ ]:
# figure for bandstructure

fig, ax = plt.subplots()
# specify horizontal axis details
# set range of horizontal axis
ax.set_xlim(k_node[0],k_node[-1])
# put tickmarks and labels at node positions
ax.set_xticks(k_node)
ax.set_xticklabels(label)
# add vertical lines at node positions
for n in range(len(k_node)):
  ax.axvline(x=k_node[n],linewidth=0.5, color='k')
# put title
ax.set_title("SSH band structure")
ax.set_xlabel("Path in k-space")
ax.set_ylabel("Band energy")

# plot first and second band
ax.plot(k_dist,evals_bnd[0])
ax.plot(k_dist,evals_bnd[1])

# tight layout
fig.tight_layout()

### Calculate density of states

In [ ]:
# first solve the model on a mesh and return all energies
kmesh=500
kpts=[]
for i in range(kmesh):
    kpts.append([-0.5+float(i)/float(kmesh)])
# solve the model on this mesh
evals_all=my_model.solve_all(kpts)
# flatten completely the matrix
evals_all_flat=evals_all.flatten()

### Plot density of states

In [ ]:
fig2, ax2 = plt.subplots()
# number of intervals
nint=201
ax2.hist(evals_all_flat,nint,range=(-2.,2.), orientation='horizontal')
ax2.set_ylim(-2.,2.)
# put title
ax2.set_title("SSH density of states")
ax2.set_xlabel("Number of states")
ax2.set_ylabel("Band energy")
# make an PDF figure of a plot
fig2.tight_layout()

### Plot DOS together with the bands

In [ ]:
v=0.5
w=1.0

my_model=tb_model(1,1,lat,orb)
my_model.set_onsite([0,0])
my_model.set_hop(v, 0, 1, [0])
my_model.set_hop(w, 1, 0, [1])
(evals_bnd,evecs_bnd)=my_model.solve_all(k_vec,eig_vectors=True)


fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True,figsize = (12,6))
ax1.set_xlim(k_node[0],k_node[-1])
ax1.set_xticks(k_node)
ax1.set_xticklabels(label)
for n in range(len(k_node)):
  ax1.axvline(x=k_node[n],linewidth=0.5, color='k')
ax1.set_title("SSH band structure")
ax.set_xlabel("Path in k-space")
ax1.set_ylabel("Band energy")
ax1.plot(k_dist,evals_bnd[0])
ax1.plot(k_dist,evals_bnd[1])


kmesh=500
kpts=[]
for i in range(kmesh):
    kpts.append([-0.5+float(i)/float(kmesh)])
evals_all=my_model.solve_all(kpts)
evals_all_flat=evals_all.flatten()

nint=201
ax2.hist(evals_all_flat,nint,range=(-2.,2.), orientation='horizontal')
ax2.set_ylim(-2.,2.)
ax2.set_title("SSH density of states")
ax2.set_xlabel("Number of states")

fig.tight_layout()
plt.show()

Note the peaks at the band edges. DOS in 1D: $D(\epsilon) \propto 1/\sqrt{\epsilon}$

### Calculate the velocities along the path

$v_n(k_i) = \frac{1}{\hbar} \frac{\partial \epsilon_n (k_i)}{\partial k_i} \approx \frac{1}{\hbar} \frac{ \epsilon_n (k_{i+1}) - \epsilon_n (k_{i}) }{k_{i+1}-k_i} $

In [ ]:
vel = np.zeros((2,len(k_dist)-1))
for i in range(len(k_dist)-1):
    for m in range(2):
        vel[m,i] = (evals_bnd[m,i+1]-evals_bnd[m,i])/(k_dist[i+1]-k_dist[i])

### Plot the velocities

In [ ]:
# Create the colormap
colors = [(0,0,1),(1, 1, 1),(1,0,0)]  # B -> W -> R
cmap_name = 'my_list'
cm = LinearSegmentedColormap.from_list(cmap_name, colors, 100)

fig, ax = plt.subplots()

# specify horizontal axis details
# set range of horizontal axis
ax.set_xlim(k_node[0],k_node[-1])
# put tickmarks and labels at node positions
ax.set_xticks(k_node)
ax.set_xticklabels(label)
# add vertical lines at node positions
for n in range(len(k_node)):
  ax.axvline(x=k_node[n],linewidth=0.5, color='k')
# put title
ax.set_title("SSH band structure")
ax.set_xlabel("Path in k-space")
ax.set_ylabel("Band energy")

# plot first and second band
bc=ax.scatter(k_dist[:-1],evals_bnd[0,:-1],c=vel[0],s=5,cmap=cm,vmin=-10, vmax=10)
bc=ax.scatter(k_dist[:-1],evals_bnd[1,:-1],c=vel[1],s=5,cmap=cm,vmin=-10, vmax=10)
cbar=plt.colorbar(bc,ticks=[-10,0.0,10], shrink=0.9, pad = 0.1)
cbar.ax.set_yticklabels([r"$-10$",r"$0$",r"$10$"],size=20)
cbar.ax.set_ylabel(r"$\hbar v$", rotation=270,size=20)

# tight layout
fig.tight_layout()
plt.show()

### Electronic structure of a finite chain

large number of cells

In [ ]:
# cutout finite model along direction 0
cut_finite=my_model.cut_piece(20,0,glue_edgs=False)
cut_periodic=my_model.cut_piece(20,0,glue_edgs=True)

### Visualize

In [ ]:
(fig,ax)=cut_finite.visualize(0)
ax.set_title("SSH, chain")
ax.set_xlabel("x coordinate")
ax.set_ylabel("y coordinate")
fig.tight_layout()

### Electronic structure of the chain

- not periodic
- discrete levels

In [ ]:
v=0.5
w=1

my_model=tb_model(1,1,lat,orb)
my_model.set_onsite([0,0])
my_model.set_hop(v, 0, 1, [0])
my_model.set_hop(w, 1, 0, [1])
(evals_bnd,evecs_bnd)=my_model.solve_all(k_vec,eig_vectors=True)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True,figsize = (12,6))
ax1.set_xlim(k_node[0],k_node[-1])
ax1.set_xticks(k_node)
ax1.set_xticklabels(label)
for n in range(len(k_node)):
  ax1.axvline(x=k_node[n],linewidth=0.5, color='k')
ax1.set_title("SSH band structure")
ax.set_xlabel("Path in k-space")
ax1.set_ylabel("Band energy")
ax1.plot(k_dist,evals_bnd[0])
ax1.plot(k_dist,evals_bnd[1])


cut_finite=my_model.cut_piece(100,0,glue_edgs=False)
cut_periodic=my_model.cut_piece(100,0,glue_edgs=True)
(evals_per,evecs_per)=cut_periodic.solve_all(eig_vectors=True)
(evals_fin,evecs_fin)=cut_finite.solve_all(eig_vectors=True)

nint=201
ax2.hist(evals_per,nint,range=(-2.,2.), orientation='horizontal')
ax2.set_ylim(-2.,2.)
ax2.set_title("SSH density of states - periodic chain")
ax2.set_xlabel("Number of states")

ax3.hist(evals_fin,nint,range=(-2.,2.), orientation='horizontal')
ax3.set_ylim(-2.,2.)
ax3.set_title("SSH density of states - finite chain")
ax3.set_xlabel("Number of states")

fig.tight_layout()
plt.show()

### Localization of the edge state

In [ ]:
cut_finite=my_model.cut_piece(10,0,glue_edgs=False)
(evals_fin,evecs_fin)=cut_finite.solve_all(eig_vectors=True)

# pick index of state in the middle of the gap
ed=cut_finite.get_num_orbitals()//2

# draw one of the edge states
(fig,ax)=cut_finite.visualize(0,eig_dr=evecs_fin[ed,:])
ax.set_title("Edge state")
ax.set_xlabel("x coordinate")
ax.set_ylabel("y coordinate")
fig.tight_layout()


### Localization of a bulk state

In [ ]:
cut_finite=my_model.cut_piece(10,0,glue_edgs=False)
(evals_fin,evecs_fin)=cut_finite.solve_all(eig_vectors=True)

# pick index of state in the middle of the gap
ed=cut_finite.get_num_orbitals()//4

# draw one of the edge states
(fig,ax)=cut_finite.visualize(0,eig_dr=evecs_fin[ed,:])
ax.set_title("Edge state")
ax.set_xlabel("x coordinate")
ax.set_ylabel("y coordinate")
fig.tight_layout()

### Edge states on the boundary between a topological and a normal insulator

In [ ]:
# length of the TI
lti = 8
# length of the NI
lni = 4
# total length
l = lti+lni

# define lattice vectors
lat=[[l]]
# define coordinates of orbitals
orb = []
for i in range(l):
    orb.append([(i+0.25)/l])
    orb.append([(i+0.75)/l])

### Create the model

In [ ]:
# create the SSH model
my_model=tb_model(1,1,lat,orb)

# set model parameters
v=0.5
w=1.0
#v_per = 0.
v_per = v

# set on-site energies
zeros = np.zeros(len(orb))
my_model.set_onsite(zeros.tolist())
# set hoppings (one for each connected pair of orbitals)
# (amplitude, i, j, [lattice vector to cell containing j])
# TI
for i in range(lti):
    my_model.set_hop(v, 2*i, 2*i+1, [0])
    my_model.set_hop(w, 2*i+1, 2*i+2, [0])
# NI
for i in range(lti,l-1):
    my_model.set_hop(w, 2*i, 2*i+1, [0])
    my_model.set_hop(v, 2*i+1, 2*i+2, [0])
# last cell (periodic?)
i = l-1
my_model.set_hop(w, 2*i, 2*i+1, [0])
my_model.set_hop(v_per, 2*i+1, 0, [1])

# print tight-binding model
#my_model.display()

In [ ]:
my_model.visualize(0)

In [ ]:
path=[[-0.5],[0.0],[0.5]]
# labels of the nodes
label=(r'$-X$',r'$\Gamma $',r'$X$')
# total number of interpolated k-points along the path
nk=101

# call function k_path to construct the actual path
(k_vec,k_dist,k_node)=my_model.k_path(path,nk)

(evals_bnd,evecs_bnd)=my_model.solve_all(k_vec,eig_vectors=True)

fig, (ax1,ax2,ax3) = plt.subplots(1,3,figsize = (15,6))
ax1.set_xlim(k_node[0],k_node[-1])
ax1.set_xticks(k_node)
ax1.set_xticklabels(label)
for n in range(len(k_node)):
  ax1.axvline(x=k_node[n],linewidth=0.5, color='k')
ax1.set_title("SSH superlattice (periodic) band structure")
ax1.set_xlabel("Path in k-space")
ax1.set_ylabel("Band energy")
for ib in range(len(orb)):
    ax1.plot(k_dist,evals_bnd[ib], color = 'k')

ax2.set_xlim(k_node[0],k_node[-1])
ax2.set_ylim(-0.05,0.05)
ax2.set_xticks(k_node)
ax2.set_xticklabels(label)
for n in range(len(k_node)):
  ax2.axvline(x=k_node[n],linewidth=0.5, color='k')
ax2.set_title("SSH superlattice (periodic) band structure")
ax2.set_xlabel("Path in k-space")
#ax2.set_ylabel("Band energy")
for ib in range(len(orb)):
    ax2.plot(k_dist,evals_bnd[ib], color = 'k')
    
cut_finite=my_model.cut_piece(5,0,glue_edgs=False)
(evals_fin,evecs_fin)=cut_finite.solve_all(eig_vectors=True)
nint=51
ax3.hist(evals_fin,nint,range=(-0.05,0.05), orientation='horizontal')
ax3.set_ylim(-0.05,0.05)
ax3.set_title("SSH superlattice (finite, 5 SL cells) DOS")
ax3.set_xlabel("Number of states")

### Localization of interface and edge states

In [ ]:
# pick index of state in the middle of the gap
ed=cut_finite.get_num_orbitals()//2-2

# draw one of the edge states
(fig,ax)=cut_finite.visualize(0,eig_dr=evecs_fin[ed,:])
ax.set_title("Interface state",size = 20)
ax.set_xlabel("x coordinate")
ax.set_ylabel("y coordinate")
fig.tight_layout()

# pick index of state in the middle of the gap
ed=cut_finite.get_num_orbitals()//2

# draw one of the edge states
(fig,ax)=cut_finite.visualize(0,eig_dr=evecs_fin[ed,:])
ax.set_title("Edge state",size = 20)
ax.set_xlabel("x coordinate")
ax.set_ylabel("y coordinate")
fig.tight_layout()